In [9]:
from sentence_transformers import SentenceTransformer

/Users/amoghkulkarni/.local/share/virtualenvs/python_practice-ZCCdr_2d/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
model_name = 'multi-qa-distilbert-cos-v1'

In [11]:
model = SentenceTransformer(model_name)

In [17]:
user_question = "I just discovered the course. Can I still join it?"

In [18]:
v = model.encode(user_question)

In [37]:
len(v)

768

In [2]:
import json

In [3]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [4]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'id': '1f6520ca'}

In [5]:
from tqdm.auto import tqdm

In [6]:
import numpy as np

In [25]:
import pandas as pd

In [12]:
embeddings = []

for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(model.encode(qa_text))

  0%|          | 0/948 [00:00<?, ?it/s]

In [13]:
X = np.array(embeddings)

In [14]:
X.shape

(948, 768)

In [19]:
scores = X.dot(v)

In [21]:
max(scores)

0.6506574

In [22]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [ ]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

In [26]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [27]:
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']

In [28]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [29]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [30]:
    ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [31]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'id': '1f6520ca'}

In [33]:
relevance_total = []

for doc in tqdm(ground_truth):
    doc_id = doc['document']
    q_v = model.encode(doc['question'])
    results = search_engine.search(q_v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [34]:
hit_rate(relevance_total)

0.9218579234972678

In [36]:
for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    doc['question_text_vector']= model.encode(qa_text)

  0%|          | 0/948 [00:00<?, ?it/s]

In [38]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [39]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [40]:
user_question = "I just discovered the course. Can I still join it?"

In [42]:
v_q = model.encode(user_question)

In [43]:
search_query = {
    "field": "question_text_vector",
    "query_vector": v_q,
    "k": 5,
    "num_candidates": 10000
}

In [44]:
es_results = es_client.search(
    index=index_name,
    knn=search_query,
    source=["text", "section", "question", "course", "id"]
)

In [ ]:
es_results['hits']['hits']

In [61]:
def es_client_search_eval(query_vector):
    
    search_query = {
        "field": "question_text_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 10000
    }

    es_results = es_client.search(
        index=index_name,
        knn=search_query,
        source=["text", "section", "question", "course", "id"]
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [51]:
ground_truth[1]

{'question': 'Can you provide a link to sign up?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [66]:
def hit_rate_evaluate():
    relevance_total = []

    for doc in tqdm(ground_truth):
        doc_id = doc['document']
        v_q = model.encode(doc['question'])
        results = es_client_search_eval(v_q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return hit_rate(relevance_total)

In [58]:
test_v_q = model.encode(ground_truth[0]['question'])

In [ ]:
es_client_search_eval(test_v_q)

In [67]:
hit_rate_evaluate()

  0%|          | 0/1830 [00:00<?, ?it/s]

0.9218579234972678